In [155]:
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf



## For testing purposes

In [123]:
testing_ds = tf.data.Dataset.range(10)
testing_ds = testing_ds.shuffle(20, reshuffle_each_iteration=True)
print(list(testing_ds.as_numpy_iterator()))
print(list(testing_ds.as_numpy_iterator()))
train_ds = testing_ds.take(8)
test_ds = testing_ds.take(2)

print(list(train_ds.as_numpy_iterator()))
print(list(test_ds.as_numpy_iterator()))
print(list(test_ds.as_numpy_iterator()))
print(list(test_ds.as_numpy_iterator()))




## Loading the data

In [350]:

import tensorflow as tf

# Parameters
batch_size = 32
img_height = 140
img_width = 140
images_dir = "/home/mihas/KNU/4Grade1Semester/ML/season_photo_classificator/images"

# Load the original dataset with shuffling enabled
train_ds, test_ds = tf.keras.utils.image_dataset_from_directory(
    images_dir,
    validation_split=0.2,
    subset="both",
    shuffle=True,
    seed=123,
    image_size=(img_height, img_width),
    batch_size=None
)

normalization_layer = layers.Rescaling(1./255)

train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
test_ds = test_ds.map(lambda x, y: (normalization_layer(x), y))
test_ds = test_ds.batch(batch_size)

images = []
labels = []
for image, label in train_ds:
    images.append(image)
    labels.append(label)

train_size = int(0.9 * len(images))
val_size = int(0.1 * len(images))

train_images, val_images = images[val_size:], images[:val_size]
train_labels, val_labels = labels[val_size:], labels[:val_size]

train_ds = tf.data.Dataset.from_tensor_slices((train_images, train_labels)).shuffle(1000).batch(batch_size)
val_ds = tf.data.Dataset.from_tensor_slices((val_images, val_labels)).batch(batch_size)

data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomRotation(0.1),
    tf.keras.layers.RandomZoom(0.2),
    tf.keras.layers.RandomContrast(0.2),
    tf.keras.layers.RandomFlip("horizontal")
])
train_ds = train_ds.map(lambda x, y: (data_augmentation(x, training=True), y))

print(f"Train size {len(train_ds)*batch_size}, validation size {len(val_ds)*batch_size}, test size {len(test_ds)*batch_size}")

AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)


In [ ]:
print(next(iter(test_ds)))

## View some samples from dataseе

In [343]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

## Creating the model

In [351]:
from tensorflow.keras import layers

num_classes = 4

model = tf.keras.Sequential(
    [
        layers.Input([img_width, img_height, 3]),
        layers.Conv2D(64, 8, strides=3, activation="relu", padding="same"),
        layers.MaxPooling2D(2),
        layers.Conv2D(128, 5, activation="relu", padding="same"),
        layers.Conv2D(128, 5, activation="relu", padding="same"),
        layers.MaxPooling2D(2),
        layers.Conv2D(258, 3, activation="relu", padding="same"),
        layers.Conv2D(256, 3, activation="relu", padding="same"),
        layers.MaxPooling2D(2),
        layers.Flatten(),
        layers.Dense(128, activation="relu"),
        layers.Dropout(0.5),
        layers.Dense(64, activation="relu"),
        layers.Dropout(0.5),
        layers.Dense(10, activation="softmax"),
    ]
)

In [275]:
model.summary()

In [352]:
model.compile(
  optimizer='adam',
  loss=tf.keras.losses.SparseCategoricalCrossentropy(),
  metrics=['accuracy'])


In [358]:
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model


checkpoint_cb = ModelCheckpoint(
    "/home/mihas/KNU/4Grade1Semester/ML/season_photo_classificator/season_classifier/best_model.keras",
    monitor="val_accuracy",
    save_best_only=True,
    mode="max",
)

history = model.fit(
    train_ds, validation_data=val_ds, epochs=5, callbacks=[checkpoint_cb]
)


best_model = load_model(
    "/home/mihas/KNU/4Grade1Semester/ML/season_photo_classificator/season_classifier/best_model.keras"
)

## Checking model metrics

### Training history

In [359]:

import pandas as pd
import matplotlib.pyplot as plt

print(f"Best val accuracy {max(history.history["val_accuracy"])}")

pd.DataFrame(history.history).plot(figsize=(8, 5))
plt.grid(True)
plt.gca().set_ylim(0, 2)
plt.show()


### Confusion matrix

In [363]:
import seaborn as sns
import matplotlib.pyplot as plt

y_pred = model.predict(test_ds)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.concatenate([y for x, y in test_ds], axis=0)

accuracy = np.mean(y_true == y_pred_classes)
print(accuracy)

cm = tf.math.confusion_matrix(y_true, y_pred_classes)

# to remove diagonal elements
# cm = cm.numpy()  
# np.fill_diagonal(cm, 0)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='binary', xticklabels=class_names, yticklabels=class_names)
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")

plt.xticks(rotation=45, ha='left')
plt.gca().xaxis.set_ticks_position('top')  # Set x-ticks to top
plt.show()